In [25]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun

In [9]:
f = open("./openai_key")
openai_key = f.read().strip()

import os

os.environ["OPENAI_API_KEY"] = openai_key

In [10]:
llm = OpenAI(temperature=0)

In [10]:
search = DuckDuckGoSearchRun()
    
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [12]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [13]:
agent.run("How much time will it take from Wolli creek to Randwick in Sydney? ")



> Entering new AgentExecutor chain...
 I need to find out the distance and the average speed of the journey
Action: Search
Action Input: Distance and average speed from Wolli Creek to Randwick
Observation: Wolli Creek to Randwick by train and tram. The journey time between Wolli Creek and Randwick is around 33 min and covers a distance of around 13 km. This includes an average layover time of around 2 min. Operated by Sydney Trains and Transport for NSW, the Wolli Creek to Randwick service departs from Wolli Creek Station and arrives in Randwick ... Average speed is measured in units of distance per time, and the average speed formula looks like this: average speed = total distance / total time. Typical units are kilometers per hour (kph), miles per hour (mph), meters per second (m/s) and feet per second (ft/s). In our mph calculator the default unit is mph (kph for metric-units countries ... Distance Randwick Gold Coast. 690 km. Distance Randwick Canberra. 247 km. Distance Randwick 

'The journey from Wolli Creek to Randwick will take around 33 minutes and cover a distance of 13 km, with an average speed of around 24 km/h.'

In [15]:
agent.run("What is the age of the Sydney Opera House? ")



> Entering new AgentExecutor chain...
 I should look for information about the Sydney Opera House
Action: Search
Action Input: Sydney Opera House age
Observation: The Sydney Opera House is a multi-venue performing arts centre in Sydney.Located on the foreshore of Sydney Harbour, it is widely regarded as one of the world's most famous and distinctive buildings and a masterpiece of 20th-century architecture.. Designed by Danish architect Jørn Utzon, but completed by an Australian architectural team headed by Peter Hall, the building was formally opened ... The Sydney Opera House is a multipurpose performing arts facility located on Port Jackson in Sydney, New South Wales, Australia. It was designed by Danish architect Jørn Utzon, the winner of an international competition sponsored by the New South Wales government for the complex's design. The Opera House, which was designated a UNESCO World Heritage site in 2007, is Sydney's most prominent ... Sydney Opera House under construction, 1

'The Sydney Opera House was officially opened on October 20, 1973.'

In [11]:
from langchain import LLMMathChain
llm_math_chain = LLMMathChain.from_llm(llm, verbose=True)


In [19]:
 tools = [
      Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
     
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]
    
   

In [20]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("What is the age of the Sydney Opera House? ")



> Entering new AgentExecutor chain...
 I need to find out the age of the Sydney Opera House
Action: Search
Action Input: "age of Sydney Opera House"
Observation: The Sydney Opera House is a multi-venue performing arts centre in Sydney.Located on the foreshore of Sydney Harbour, it is widely regarded as one of the world's most famous and distinctive buildings and a masterpiece of 20th-century architecture.. Designed by Danish architect Jørn Utzon, but completed by an Australian architectural team headed by Peter Hall, the building was formally opened ... The Sydney Opera House is one of the 20th century's most iconic buildings. It broke new ground for design and engineering around the world. Since its opening in 1973 it has become a symbol the world immediately associates with Sydney and Australia. The story of the Opera House is a drama that for more than 15 years grabbed national headlines ... January 31, 2023 Photo: Getty Images Sydney Opera House, originally designed by Danish arc

'The Sydney Opera House is 50 years old, having been opened in 1973.'

Notes:
    
    - This does not work very well in the HuggingFace Hub models. The LLM is not able to parse the search result and include it in its output.
    - But... can be done! :) 

In [19]:
from langchain import PromptTemplate, LLMChain, HuggingFaceHub

f = open("./hfhub_key")
hfhub_key = f.read().strip()

import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfhub_key
repo_id = "google/flan-t5-xxl"

In [20]:
template = """Question: {question}


Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_step_by_step = HuggingFaceHub(
   repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 128})
llm_chain_step_by_step = LLMChain(prompt=prompt, llm=llm_step_by_step)

parameter = "Who is the Prime Minister of Australia? "
llm_chain_step_by_step.run(parameter)

'Australia has a leader called the Prime Minister. The current Prime Minister of Australia is Malcolm Turnbull'

In [26]:
search = DuckDuckGoSearchRun()
    
def get_search_snippet(question):
    return search.run(question)

In [27]:
prompt = PromptTemplate(
    template=""""Answer the question by extracting information from the document.
    
    Document: {document}
    Question: {question}
    
    Answer: Let's think step by step. """, 
    input_variables=["document", "question"]
);



In [28]:
llm_chain_with_search_hfhub = LLMChain(prompt=prompt, llm=llm_step_by_step)

In [29]:
question = "Who is the Prime Minister of Australia? "
llm_chain_with_search_hfhub.run({"document": get_search_snippet(question), "question": question})


'The relevant information is: The current prime minister is Anthony ... Anthony Albanese, in full'